In [1]:
import os
import time
import math
import copy
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import integrate, interpolate, stats
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.integrate import quad

import healpy as hp
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import Planck18 as cosmo
from astropy.table import Table

from nbodykit.lab import *
from nbodykit import setup_logging, style

from pypower import CatalogMesh, MeshFFTPower, CatalogFFTPower, PowerSpectrumStatistics, utils, PowerSpectrumWedges
import sympy as sp
from sympy import Symbol

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [2]:
plt.rcParams["figure.figsize"] = (12,7)
plt.rcParams["font.size"] = 20
plt.rcParams["font.family"]='serif'
plt.rcParams['text.usetex']=True
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['figure.dpi'] = 600

# Functions used for modeling 


In [ ]:
def skymap(cat, nside=8):
    """
    Generates a Healpix map with redshift as values for each pixel.
    This is useful for visualizing the redshift distribution of sources across the sky.
    
    Parameters:
        cat: catalog of Right Ascension and Declination in degrees.
        nside: resolution.
        
    Returns:
        m (np.ndarray): 1D array with redshift values for each HEALPix pixel.
    
    """
    # Convert RA/Dec to galactic coordinates for uniform sky mapping
    c = SkyCoord(ra=np.asarray(cat.ra) * u.degree, dec=np.asarray(cat.dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value
    
    # HEALPix expects theta/phi in radians
    theta = np.radians(90. - b)
    phi = np.radians(l)
    
    # Assign each object to a HEALPix pixel
    pixel_indices = hp.ang2pix(nside, theta, phi)
    
    # Initialize map: each pixel will hold the redshift of the object(s) in that pixel
    m = np.zeros(hp.nside2npix(nside))
    
    m[pixel_indices] = cat.z  # This will overwrite if multiple objects fall in the same pixel
    
    return m

def countmap(Ra, Dec, nside):
    """
    Create a HEALPix map counting the number of objects per pixel from RA/Dec coordinates.

    Parameters:
        Ra (array-like): Right Ascension values in degrees.
        Dec (array-like): Declination values in degrees.
        nside (int): HEALPix NSIDE parameter (resolution).

    Returns:
        hpx_map (np.ndarray): 1D array with counts per HEALPix pixel.
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(Ra) * u.degree, dec=np.asarray(Dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value

    # Convert to HEALPix theta/phi
    theta = np.radians(90. - b)
    phi = np.radians(l)

    # Number of pixels in the map
    npix = hp.nside2npix(nside)

    # Get HEALPix pixel indices for each object
    indices = hp.ang2pix(nside, theta, phi)
    idx, counts = np.unique(indices, return_counts=True)

    # Create map and fill with counts
    hpx_map = np.zeros(npix)
    hpx_map[idx] = counts
    return hpx_map

def indices(Ra, Dec, nside=256):
    """
    Get HEALPix pixel indices for given RA/Dec coordinates.

    Parameters:
        Ra (array-like): Right Ascension values in degrees.
        Dec (array-like): Declination values in degrees.
        nside (int): HEALPix NSIDE parameter (default 256).

    Returns:
        indices (np.ndarray): Array of HEALPix pixel indices.
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(Ra) * u.degree, dec=np.asarray(Dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value

    # Convert to HEALPix theta/phi
    theta = np.radians(90. - b)
    phi = np.radians(l)

    # Get HEALPix pixel indices
    indices = hp.ang2pix(nside, theta, phi)
    return indices

def weighted_countmap(Ra, Dec, weights, nside=256):
    """
    Create a HEALPix map summing weights per pixel from RA/Dec coordinates.

    Parameters:
        Ra (array-like): Right Ascension values in degrees.
        Dec (array-like): Declination values in degrees.
        weights (array-like): Weights for each object.
        nside (int): HEALPix NSIDE parameter (default 256).

    Returns:
        hpx_map (np.ndarray): 1D array with weighted sum per HEALPix pixel.
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(Ra) * u.degree, dec=np.asarray(Dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value

    # Convert to HEALPix theta/phi
    theta = np.radians(90. - b)
    phi = np.radians(l)

    # Number of pixels in the map
    npix = hp.nside2npix(nside)

    # Get HEALPix pixel indices for each object
    indices = hp.ang2pix(nside, theta, phi)

    # Initialize the HEALPix map
    hpx_map = np.zeros(npix)

    # Sum weights for each pixel
    for pix, w in zip(indices, weights):
        hpx_map[pix] += w

    return hpx_map


def f_gotmap(Ra,Dec,weights,nside=256):
    """
    Calculate the average weight map (f_gotmap) from RA/Dec coordinates.

    Parameters:
        Ra (array-like): Right Ascension values in degrees.
        Dec (array-like): Declination values in degrees.
        weights (array-like): Weights for each object.
        nside (int): HEALPix NSIDE parameter (default 256).

    Returns:
        ave (np.ndarray): 1D array with average weights per HEALPix pixel.
    """
    # Calculate the average weight map
    ave = weighted_countmap(Ra,Dec,weights, nside =256) / countmap(Ra,Dec, nside =256)
    return ave

In [ ]:
def densitymap(cat, npix_nz, nside=8):
    """
    Create a HEALPix density map from a catalog of objects.

    Parameters:
        cat (pandas.DataFrame): Catalog with 'ra' and 'dec' columns (in degrees).
        npix_nz (int): Number of non-zero pixels (used for normalization).
        nside (int): HEALPix NSIDE parameter (default 8).

    Returns:
        hpx_map (np.ndarray): 1D array with density contrast per HEALPix pixel.
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(cat.ra) * u.degree, dec=np.asarray(cat.dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value
    theta = np.radians(90. - b)
    phi = np.radians(l)

    # Number of pixels in the map
    npix = hp.nside2npix(nside)

    # Get HEALPix pixel indices for each object
    pixel_indices = hp.ang2pix(nside, theta, phi)
    N_z = len(cat)
    N_bar = N_z / npix_nz  # Mean number per pixel
    print(N_z, npix_nz, N_bar)

    indices = hp.ang2pix(nside, theta, phi)
    idx, counts = np.unique(indices, return_counts=True)

    # Initialize the map with -1 for empty regions
    hpx_map = -np.ones(npix)
    # Fill with density contrast (counts/N_bar - 1)
    hpx_map[idx] = counts / N_bar - 1
    return hpx_map

In [ ]:
H_0 = 70  # Hubble constant in km/s/Mpc
H_0_inv = 13.98e9  # Inverse Hubble constant in years
# If we ever want to plot P_t_d, make sure to change this appropriately to the cosmology

def normalize_pdf(pdf, x_arr):
    """
    Normalize a probability density function (PDF) over a given array of x values.

    Parameters:
        pdf (array-like): Probability density values.
        x_arr (array-like): Corresponding x values.

    Returns:
        np.ndarray: Normalized PDF.
    """
    normalization = np.trapz(pdf, x_arr)  # Integrate PDF over x_arr
    return pdf / normalization

def rejection_sampling(N, p, x_low, x_high, y_max, p_args=[]):
    """
    Perform rejection sampling to draw samples from a probability distribution.

    Parameters:
        N (int): Number of samples to generate.
        p (callable): Probability distribution function to sample from.
        x_low (float): Lower bound of x range.
        x_high (float): Upper bound of x range.
        y_max (float): Maximum value of the PDF in the range (can be an overestimate).
        p_args (list): Additional arguments for the PDF function.

    Returns:
        np.ndarray: Array of sampled points.
    """
    sampled_points = []
    for i in range(N):
        accepted = False
        while not accepted:
            x = np.random.uniform(x_low, x_high)
            y = np.random.uniform(0, y_max)
            p_x = p(x, *p_args)
            if y <= p_x:
                accepted = True
                sampled_points.append(x)
    return np.array(sampled_points)

def rejection_sampling_numeric(N, m_array, p_m_array):
    """
    Perform rejection sampling using a numeric PDF array instead of a function.

    Parameters:
        N (int): Number of samples to generate.
        m_array (array-like): Sorted array of x values.
        p_m_array (array-like): PDF values corresponding to m_array.

    Returns:
        np.ndarray: Array of sampled points.
    """
    y_max = max(p_m_array)
    x_low, x_high = min(m_array), max(m_array)
    sampled_points = []
    for i in range(N):
        accepted = False
        while not accepted:
            x = np.random.uniform(x_low, x_high)
            y = np.random.uniform(0, y_max)
            # Find PDF value for x using searchsorted
            p_x = p_m_array[np.searchsorted(m_array, x, side="left")]
            if y <= p_x:
                accepted = True
                sampled_points.append(x)
    return np.array(sampled_points)

def P_t_d(t_d, kappa=1, t_min=100e6, norm=True):
    """
    Delay time distribution: P(t_d) ∝ t_d^(-kappa) for t_d >= t_min, 0 otherwise.

    Parameters:
        t_d (float or array): Delay time(s) in years.
        kappa (float): Power-law index.
        t_min (float): Minimum delay time (years).
        norm (bool): If True, normalize the distribution.

    Returns:
        np.ndarray: Probability density values.
    """
    f_x = 0
    if norm:
        # Normalized power-law with cutoff at t_min
        f_x = np.piecewise(t_d, [t_d < t_min, t_d >= t_min], [0, lambda t_d: 1/(np.log(H_0_inv/t_min)) * t_d**(-kappa)])
    else:
        f_x = np.piecewise(t_d, [t_d < t_min, t_d >= t_min], [0, lambda t_d: t_d**(-kappa)])
    return f_x

def SFR(z):
    """
    Madau-Dickinson star formation rate (SFR) as a function of redshift z.

    Parameters:
        z (float or array): Redshift(s).

    Returns:
        float or np.ndarray: SFR in units of M_sun yr^-1 Mpc^-3.
    """
    return 0.015 * (1+z)**(2.7) / (1 + ((1+z)/2.9)**(5.6)) * 1e9

def R_integrand(z, t_merg, t_min=100e6, kappa=1):
    """
    Integrand for merger rate calculation as a function of redshift.

    Parameters:
        z (float): Redshift.
        t_merg (float): Merger time in years.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index for delay time distribution.

    Returns:
        float: Value of the integrand.
    """
    t_d = cosmo.lookback_time(z).to_value('year') - t_merg
    return P_t_d(t_d, kappa=kappa, t_min=t_min) * cosmo.lookback_time_integrand(z) * SFR(z)

# def merger_rate(z, t_min=500e6, kappa=1):
#     return integrate.quad(R_integrand, z, 1e4, args=( cosmo.lookback_time(z).to_value('year'), t_min, kappa ), limit=300)[0]

# def merger_rate(z_arr, t_min=500e6, kappa=1):
#     '''
#     Computes merger rate ... Can optimize this code
#     '''
#     if isinstance(z_arr, np.ndarray):
#         R_merger = []
#         for z in z_arr:
#             R_merger.append((integrate.quad(R_integrand, z, 1e4, args=( cosmo.lookback_time(z).to_value('year'), 500e6,), limit=300))[0])
#         R_merger = np.array(R_merger)
#         return R_merger
#     return integrate.quad(R_integrand, z_arr, 1e4, args=( cosmo.lookback_time(z_arr).to_value('year'), t_min, kappa ), limit=300)[0]


def merger_rate(z, t_min=500e6, kappa=1):
    """
    Compute the merger rate at a given redshift by integrating the delay time distribution.

    Parameters:
        z (float): Redshift.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        float: Merger rate.
    """
    return integrate.quad(R_integrand, z, np.inf, args=(cosmo.lookback_time(z).to_value('year'), t_min, kappa), limit=300)[0]

def N_GW_integrand(z, interp_rate, obs_time, t_min=500e6, kappa=1):
    """
    Integrand for the expected number of gravitational wave events as a function of redshift.

    Parameters:
        z (float): Redshift.
        interp_rate (callable): Interpolated merger rate function.
        obs_time (float): Observation time (years).
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        float: Value of the integrand.
    """
    dVdz = 4 * np.pi * cosmo.differential_comoving_volume(z).to_value('Gpc3 / sr')
    return dVdz * interp_rate(z) / (1 + z) * obs_time

def N_GW_integrand_test(z, interp_rate, obs_time, t_min=500e6, kappa=1):
    """
    Alternative integrand for the expected number of GW events, using a different rate normalization.

    Parameters:
        z (float): Redshift.
        interp_rate (callable): Interpolated merger rate function.
        obs_time (float): Observation time (years).
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        float: Value of the integrand.
    """
    dVdz = 4 * np.pi * cosmo.differential_comoving_volume(z).to_value('Gpc3 / sr')
    rate = interp_rate(z, t_min=t_min, kappa=1)
    return dVdz * rate * 23.9 / interp_rate(0, t_min=t_min, kappa=1) / (1 + z) * obs_time

def N_GW_bin(z_min, z_max, obs_time, t_min=500e6, kappa=1, grid_points=1999):
    """
    Compute the expected number of GW events in a redshift bin using an interpolated merger rate.

    Parameters:
        z_min (float): Minimum redshift.
        z_max (float): Maximum redshift.
        obs_time (float): Observation time (years).
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.
        grid_points (int): Number of grid points for interpolation.

    Returns:
        float: Expected number of GW events in the bin.
    """
    z_arr = np.logspace(-10, np.log10(10), grid_points)
    z_arr = np.insert(z_arr, 0, 0)
    merger_rate_vec = np.vectorize(merger_rate)
    rate = merger_rate_vec(z_arr, t_min=t_min, kappa=kappa)
    norm_rate = rate * 23.9 / rate[0]
    norm_rate_fn = interpolate.interp1d(z_arr, norm_rate)
    return integrate.quad(N_GW_integrand, z_min, z_max, args=(norm_rate_fn, obs_time, t_min, kappa), limit=300)[0]

def N_GW_bin_no_interp(z_min, z_max, obs_time, t_min=500e6, kappa=1, grid_points=1999):
    """
    Compute the expected number of GW events in a redshift bin without interpolation.

    Parameters:
        z_min (float): Minimum redshift.
        z_max (float): Maximum redshift.
        obs_time (float): Observation time (years).
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.
        grid_points (int): Number of grid points for interpolation.

    Returns:
        float: Expected number of GW events in the bin.
    """
    merger_rate_vec = np.vectorize(merger_rate)
    return integrate.quad(N_GW_integrand_test, z_min, z_max, args=(merger_rate_vec, obs_time, t_min, kappa), limit=300)[0]


def metallicity(C1, C2):
    """
    Compute the average metallicity from two metallicity indicators.

    Parameters:
        C1 (array-like or float): First metallicity indicator.
        C2 (array-like or float): Second metallicity indicator.

    Returns:
        np.ndarray or float: The average metallicity value(s).
    """
    # Take the mean of the two metallicity indicators
    return np.array((C1 + C2) / 2)


def P_g_full(g, power_p, power_n, Break, low_g=None, high_g=None):
    """
    Compute a broken power-law probability distribution for a galaxy property (e.g., mass, SFR, metallicity).

    The distribution follows a power law with exponent 'power_p' below the break value, and 'power_n' above it.
    Optionally, lower and upper cut-offs can be applied.

    Parameters:
        g (array-like): Property values (e.g., mass, SFR, metallicity).
        power_p (float): Power-law exponent below the break.
        power_n (float): Power-law exponent above the break.
        Break (float): Break value separating the two regimes.
        low_g (float, optional): Lower cut-off for g.
        high_g (float, optional): Upper cut-off for g.

    Returns:
        np.ndarray: Normalized probability distribution for g.
    """
    g = np.array(g)
    prob = np.zeros(len(g))
    for i in range(len(g)):
        # Below the break, use power_p exponent
        if g[i] < Break:
            prob[i] = g[i] ** power_p
        # Above the break, use power_n exponent and normalize at the break
        if g[i] >= Break:
            prob[i] = g[i] ** power_n / Break ** (power_n - power_p)
    # Apply lower and upper cut-offs if specified
    if low_g:
        prob = prob * np.array([0 if g < low_g else 1 for g in g])
    if high_g:
        prob = prob * np.array([0 if g > high_g else 1 for g in g])
    # Normalize the probability distribution
    return prob / np.sum(prob)


def P_g_unconditional(
    M, sfr, metal=None,
    m_power_p=None, m_power_n=None, m_break=None, min_m=None, max_m=None,
    SFR_power_p=None, SFR_power_n=None, SFR_break=None, min_SFR=None, max_SFR=None,
    metal_power_p=None, metal_power_n=None, metal_break=None, min_metal=None, max_metal=None
):
    """
    Compute the unconditional joint probability distribution for galaxy properties (mass, SFR, metallicity).

    The distribution is the product of individual (possibly broken power-law) distributions for each property.
    If metallicity is not provided, only mass and SFR are used.

    Parameters:
        M (array-like): Stellar mass.
        sfr (array-like): Star formation rate.
        metal (array-like, optional): Metallicity.
        m_power_p, m_power_n, m_break, min_m, max_m: Parameters for mass distribution.
        SFR_power_p, SFR_power_n, SFR_break, min_SFR, max_SFR: Parameters for SFR distribution.
        metal_power_p, metal_power_n, metal_break, min_metal, max_metal: Parameters for metallicity distribution.

    Returns:
        np.ndarray: Normalized joint probability distribution.
    """
    if metal is None:
        # Product of mass and SFR distributions
        p = P_g_full(M, m_power_p, m_power_n, m_break, min_m, max_m) * \
            P_g_full(sfr, SFR_power_p, SFR_power_n, SFR_break, min_SFR, max_SFR)
    else:
        # Product of mass, SFR, and metallicity distributions
        p = P_g_full(M, m_power_p, m_power_n, m_break, min_m, max_m) * \
            P_g_full(sfr, SFR_power_p, SFR_power_n, SFR_break, min_SFR, max_SFR) * \
            P_g_full(metal, metal_power_p, metal_power_n, metal_break, min_metal, max_metal)
    # Normalize the joint probability
    p = p / np.sum(p)
    return p


def P_MZR(M, delta_l, delta_h, M_c, low_M=None, high_M=None):
    """
    Compute the probability distribution for the Mass-Metallicity Relation (MZR).

    This is modeled as a broken power-law in stellar mass, with a break at M_c.

    Parameters:
        M (array-like): Stellar mass.
        delta_l (float): Power-law slope below the break (low mass end).
        delta_h (float): Power-law slope above the break (high mass end).
        M_c (float): Break mass.
        low_M (float, optional): Lower cut-off for mass.
        high_M (float, optional): Upper cut-off for mass.

    Returns:
        np.ndarray: Probability distribution for MZR.
    """
    # Use P_g_full with appropriate exponents for the broken power-law
    return P_g_full(M, 1/delta_l, -1/delta_h, M_c, low_M, high_M)


def P_FMR(M, sfr, delta_h, M_c, low_M, high_M, epsilon_h, sfr_c, low_sfr, high_sfr):
    """
    Compute the probability distribution for the Fundamental Metallicity Relation (FMR).

    The FMR is modeled as the product of two broken power-law distributions:
    one in mass (M) and one in SFR, each with their own break and slope.

    Parameters:
        M (array-like): Stellar mass.
        sfr (array-like): Star formation rate.
        delta_h (float): Slope above the break for mass.
        M_c (float): Break mass.
        low_M, high_M (float): Mass cut-offs.
        epsilon_h (float): Slope above the break for SFR.
        sfr_c (float): Break SFR.
        low_sfr, high_sfr (float): SFR cut-offs.

    Returns:
        np.ndarray: Probability distribution for FMR.
    """
    # Product of mass and SFR broken power-law distributions
    # Note: normalization is handled later in the workflow
    return P_g_full(M, 0, -1/delta_h, M_c, low_M, high_M) * \
           P_g_full(sfr, 0, -1/epsilon_h, sfr_c, low_sfr, high_sfr)


def P_MZSFR(
    M, sfr, Z,
    delta_l, delta_h, M_c, low_M, high_M,
    epsilon_l, epsilon_h, sfr_c, low_sfr, high_sfr,
    zeta_l, zeta_h, Z_c, low_Z, high_Z
):
    """
    Compute the joint probability distribution for the Mass-Z-SFR relation (MZSFR).

    This is modeled as the product of three broken power-law distributions:
    one each for mass (M), SFR, and metallicity (Z), with their own breaks and slopes.

    Parameters:
        M (array-like): Stellar mass.
        sfr (array-like): Star formation rate.
        Z (array-like): Metallicity.
        delta_l, delta_h (float): Slopes for mass below/above break.
        M_c (float): Break mass.
        low_M, high_M (float): Mass cut-offs.
        epsilon_l, epsilon_h (float): Slopes for SFR below/above break.
        sfr_c (float): Break SFR.
        low_sfr, high_sfr (float): SFR cut-offs.
        zeta_l, zeta_h (float): Slopes for Z below/above break.
        Z_c (float): Break metallicity.
        low_Z, high_Z (float): Metallicity cut-offs.

    Returns:
        np.ndarray: Probability distribution for MZSFR.
    """
    # Product of mass, SFR, and metallicity broken power-law distributions
    # Note: normalization is handled later in the workflow
    return P_g_full(M, 1/delta_l, -1/delta_h, M_c, low_M, high_M) * \
           P_g_full(sfr, 1/epsilon_l, -1/epsilon_h, sfr_c, low_sfr, high_sfr) * \
           P_g_full(Z, 1/zeta_l, -1/zeta_h, Z_c, low_Z, high_Z)

def g_selection_full(df,N,p,p_args,SFR_Z_type):
    """
    Select a subsample of galaxies based on a probability distribution function (PDF) and given parameters.

    Parameters:
        df (DataFrame): Input data containing galaxy properties.
        N (int): Number of galaxies to select.
        p (callable): PDF function for selection.
        p_args (list): Additional arguments for the PDF function.
        SFR_Z_type (str): Type of model to use for selection ('old_powerlaw_model', 'old_powerlaw_observed', 'MZR', 'FMR', 'MZSFR').

    Returns:
        DataFrame: Subsampled data.
    """
    if SFR_Z_type=='old_powerlaw_model':
        sfr=SFR(np.array(df['z']))
        M=np.array(df['M'])
        metal=None
        prob=p(M,sfr,metal,*p_args)
        prob=prob/np.sum(prob)
    elif SFR_Z_type=='old_powerlaw_observed':
        sfr=np.array(df['sfr'])
        metal=np.array(df['metal'])
        M=np.array(df['M'])
        prob=p(M,sfr,metal,*p_args)
        prob=prob/np.sum(prob)
    elif SFR_Z_type=='MZR':
        M=np.array(df['M'])
        prob=p(M,*p_args)
        prob=prob/np.sum(prob)
    elif SFR_Z_type=='FMR':
        sfr=np.array(df['sfr'])
        M=np.array(df['M'])
        prob=p(M,sfr,*p_args)
        prob=prob/np.sum(prob)
    elif SFR_Z_type=='MZSFR':
        sfr=np.array(df['sfr'])
        metal=np.array(df['metal'])
        M=np.array(df['M'])
        prob=p(M,sfr,metal,*p_args)
        prob=prob/np.sum(prob)
        
    index=np.array(df.index)
    new_index=np.random.choice(index, size=(N), replace=False, p=prob)    
    new_df=df.loc[new_index]
    
    return new_df


# Binary Mass Modules

In [ ]:
def m_kroupa(m, m_min, m_max, m_power):
    """
    Compute the normalized Kroupa initial mass function (IMF) probability for given masses.

    Parameters:
        m (float or array-like): Mass or array of masses.
        m_min (float): Minimum mass.
        m_max (float): Maximum mass.
        m_power (float): Power-law index.

    Returns:
        np.ndarray: Probability values for each mass.
    """
    # Normalization constant for the power-law IMF
    norm = (m_max ** (-m_power + 1) - m_min ** (-m_power + 1)) / (-m_power + 1)
    p_m = np.array([])
    if np.isscalar(m):
        m = [m]
    # Loop over all masses and compute probability
    for i in range(len(m)):
        if m[i] < m_min:
            p_m = np.append(p_m, 0)
        elif m_min <= m[i] <= m_max:
            p_m = np.append(p_m, m[i] ** -m_power / norm)
        else:
            p_m = np.append(p_m, 0)
    return p_m

def m_kroupa_unnormal(m, m_min, m_max, m_power):
    """
    Compute the unnormalized Kroupa IMF probability for given masses.

    Parameters:
        m (float or array-like): Mass or array of masses.
        m_min (float): Minimum mass.
        m_max (float): Maximum mass.
        m_power (float): Power-law index.

    Returns:
        np.ndarray: Unnormalized probability values for each mass.
    """
    p_m = np.array([])
    if np.isscalar(m):
        m = [m]
    # Loop over all masses and compute unnormalized probability
    for i in range(np.size(m)):
        if m[i] < m_min:
            p_m = np.append(p_m, 0)
        elif m_min <= m[i] <= m_max:
            p_m = np.append(p_m, m[i] ** -m_power)
        else:
            p_m = np.append(p_m, 0)
    return p_m

def m_peak_model(m, m_min, m_max, m_power, peak_pos, sigma, height):
    """
    Compute a mass function with a Kroupa IMF plus a Gaussian peak.

    Parameters:
        m (float or array-like): Mass or array of masses.
        m_min (float): Minimum mass.
        m_max (float): Maximum mass.
        m_power (float): Power-law index.
        peak_pos (float): Center of the Gaussian peak.
        sigma (float): Width of the Gaussian peak.
        height (float): Height of the Gaussian peak.

    Returns:
        np.ndarray: Normalized probability values for each mass.
    """
    p_m = m_kroupa_unnormal(m, m_min, m_max, m_power) + height * 1 / (sigma * (2 * np.pi) ** 0.5) * np.exp(-0.5 * ((m - peak_pos) / sigma) ** 2)
    norm = quad(m_kroupa_unnormal, m_min, m_max, args=(m_min, m_max, m_power))[0] + height
    return p_m / norm

def sec_mass_sample(mass_sample, m_min):
    """
    Sample secondary masses for binaries uniformly in mass ratio q.

    Parameters:
        mass_sample (float or array-like): Primary mass or array of primary masses.
        m_min (float): Minimum mass for secondary.

    Returns:
        np.ndarray: Array of secondary masses.
    """
    if np.isscalar(mass_sample):
        mass_sample = [mass_sample]
    sec_mass = np.zeros(len(mass_sample))
    for i in range(len(mass_sample)):
        q = np.random.uniform(m_min / mass_sample[i], 1)
        sec_mass[i] = q * mass_sample[i]
    return sec_mass

def sec_mass_sample_beta(mass_sample, m_min, beta):
    """
    Sample secondary masses for binaries using a beta distribution in mass ratio.

    Parameters:
        mass_sample (float or array-like): Primary mass or array of primary masses.
        m_min (float): Minimum mass for secondary.
        beta (float): Power-law index for mass ratio distribution.

    Returns:
        np.ndarray: Array of secondary masses.
    """
    if np.isscalar(mass_sample):
        mass_sample = [mass_sample]
    sec_mass = np.zeros(len(mass_sample))
    for i in range(len(mass_sample)):
        p_max = find_max(m_kroupa, m_min, mass_sample[i], [m_min, mass_sample[i], -beta])
        sec_mass[i] = rejection_sampling(1, m_kroupa, m_min, mass_sample[i], p_max, [m_min, mass_sample[i], -beta])[0]
    return sec_mass

def find_max(f, x_low, x_high, args):
    """
    Find the maximum value of a function f in the interval [x_low, x_high].

    Parameters:
        f (callable): Function to maximize.
        x_low (float): Lower bound.
        x_high (float): Upper bound.
        args (list): Arguments for the function f.

    Returns:
        float: Maximum value of f in the interval.
    """
    x = np.linspace(x_low, x_high, 100)
    f_x = f(x, *args)
    return max(f_x)

def find_nearest(array, value):
    """
    Find the nearest value in an array to a given value.

    Parameters:
        array (array-like): Array to search.
        value (float): Value to find.

    Returns:
        float: Nearest value in the array.
    """
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1]
    else:
        return array[idx]

def find_nearest_idx(array, value):
    """
    Find the index of the nearest value in an array to a given value.

    Parameters:
        array (array-like): Array to search.
        value (float): Value to find.

    Returns:
        int: Index of the nearest value in the array.
    """
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    else:
        return idx

def chirp_mass(m1, m2):
    """
    Compute the chirp mass for a binary system.

    Parameters:
        m1 (float): Mass of the primary.
        m2 (float): Mass of the secondary.

    Returns:
        float: Chirp mass.
    """
    return (m1 * m2) ** (3/5) / (m1 + m2) ** (1/5)

def PISN_mass(z, Z_star, M_Z_star, alpha, gamma, zeta):
    """
    Compute the pair-instability supernova (PISN) mass as a function of redshift and metallicity.

    Parameters:
        z (float): Redshift.
        Z_star (float): Reference metallicity.
        M_Z_star (float): Reference mass.
        alpha, gamma, zeta (float): Model parameters.

    Returns:
        float: PISN mass.
    """
    Z = 10 ** (gamma * z + zeta)  # metallicity from redshift
    return M_Z_star - alpha * np.log10(Z / Z_star)

def source_win(z, m, Z_star, M_Z_star, alpha, gamma, zeta):
    """
    Source frame window function for binary formation.

    Parameters:
        z (float): Redshift.
        m (float or array-like): Mass.
        Z_star, M_Z_star, alpha, gamma, zeta: Model parameters.

    Returns:
        np.ndarray: Window function values (1 or 0).
    """
    Z = 10 ** (gamma * z + zeta)
    m_PISN = PISN_mass(z, Z_star, M_Z_star, alpha, gamma, zeta)
    w = np.piecewise(m, [m < m_PISN, m >= m_PISN], [1, 0])
    return w

def win_integrand(z, m, Z_star, M_Z_star, alpha, gamma, zeta, z_merg, t_min=100e6, kappa=1):
    """
    Integrand for the window function in the observer frame.

    Parameters:
        z (float): Redshift.
        m (float): Mass.
        Z_star, M_Z_star, alpha, gamma, zeta: Model parameters.
        z_merg (float): Merger redshift.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        float: Value of the integrand.
    """
    t_d = cosmo.lookback_time(z).to_value('year') - cosmo.lookback_time(z_merg).to_value('year')
    return P_t_d(t_d, kappa=kappa, t_min=t_min) * cosmo.lookback_time_integrand(z) * source_win(z, m, Z_star, M_Z_star, alpha, gamma, zeta)

def obs_win(m, z_merg, Z_star, M_Z_star, alpha, gamma, zeta, t_min=100e6, kappa=1):
    """
    Observer frame window function (not normalized).

    Parameters:
        m (float): Mass.
        z_merg (float): Merger redshift.
        Z_star, M_Z_star, alpha, gamma, zeta: Model parameters.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        float: Value of the window function.
    """
    return integrate.quad(win_integrand, z_merg, np.inf, args=(m, Z_star, M_Z_star, alpha, gamma, zeta, z_merg, t_min, kappa), epsabs=1.49e-13, limit=300)[0]

def obs_win_normal(m, z_merg, Z_star, M_Z_star, alpha, gamma, zeta, t_min=100e6, kappa=1):
    """
    Normalized observer frame window function.

    Parameters:
        m (array-like): Array of masses.
        z_merg (float): Merger redshift.
        Z_star, M_Z_star, alpha, gamma, zeta: Model parameters.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index.

    Returns:
        np.ndarray: Normalized window function values.
    """
    norm = integrate.quad(obs_win, min(m), max(m), args=(z_merg, Z_star, M_Z_star, alpha, gamma, zeta, t_min, kappa), epsabs=1.49e-13, limit=300)[0]
    w = np.zeros(len(m))
    for i in range(len(w)):
        w[i] = obs_win(m[i], z_merg, Z_star, M_Z_star, alpha, gamma, zeta, t_min, kappa) / norm
    return w


def binary_masses_sample(df, p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args, z_min, z_max, n_z, t_min=100e6, kappa=1):
    """
    Generate binary component masses (m1, m2) for a galaxy sample using various physical and phenomenological models.

    Parameters:
        df (DataFrame): Input data containing galaxy properties (mass, SFR, metallicity, redshift).
        p_m1_type (str): Type of primary mass model ('M_given', 'MSFR_given', 'Z_given', 'phys', 'phen').
        p_m1 (callable): PDF/function for primary mass.
        p_m1_args (list): Arguments for p_m1.
        p_m2 (callable): Function for secondary mass given m1.
        p_m2_args (list): Arguments for p_m2.
        z_min (float): Minimum redshift for window function grid.
        z_max (float): Maximum redshift for window function grid.
        n_z (int): Number of redshift grid points for window function.
        t_min (float): Minimum delay time (years).
        kappa (float): Power-law index for delay time distribution.

    Returns:
        None. Modifies df in-place by adding 'm1' and 'm2' columns.
    """
    if p_m1_type == 'M_given':
        # p_m1_args should be given in this order:
        # p_m1_args=[m_min,m_max,m_power,Z_star,M_Z_star,alpha]
        # see arXiv:2112.10256v2 for the description of above parameters.
        m_min, m_max, m_power, Z_star, M_Z_star, alpha = p_m1_args
        num = df.shape[0]
        m1 = np.zeros(num)
        m2 = np.zeros(num)
        M = np.array(df['M'])
        solar_OH = 10 ** (8.83 - 12)
        solar_metal = 0.017
        for i in range(num):
            # note that GLADE mass is given in the units of 10^10 solar mass.
            OH = 10 ** (8.96 + 0.31 * np.log10(M[i]) - 0.23 * (np.log10(M[i])) ** 2 - 0.017 * (np.log10(M[i])) ** 3 + 0.04 * (np.log10(M[i])) ** 4 - 12)
            metal = 10 ** (np.log10(solar_metal) + np.log10(OH) - np.log10(solar_OH))
            m_PISN = M_Z_star - alpha * np.log10(metal / Z_star)
            p_m1_args_new = [m_min, m_PISN, m_power]
            p_max = find_max(p_m1, m_min, m_PISN, p_m1_args_new)
            m1[i] = rejection_sampling(1, p_m1, m_min, m_max, p_max, p_m1_args_new)
            m2[i] = p_m2(m1[i], *p_m2_args)
        df.insert(df.shape[1], 'm1', m1)
        df.insert(df.shape[1], 'm2', m2)
    if p_m1_type == 'MSFR_given':
        # p_m1_args should be given in this order:
        # p_m1_args=[m_min,m_max,m_power,Z_star,M_Z_star,alpha]
        # see arXiv:2112.10256v2 for the description of above parameters.
        m_min, m_max, m_power, Z_star, M_Z_star, alpha = p_m1_args
        num = df.shape[0]
        m1 = np.zeros(num)
        m2 = np.zeros(num)
        M = np.array(df['M'])
        sfr = np.array(df['sfr'])
        solar_OH = 10 ** (8.83 - 12)
        solar_metal = 0.017
        for i in range(num):
            # note that GLADE mass is given in the units of 10^10 solar mass.
            OH = 10 ** (8.96 + 0.37 * np.log10(M[i]) - 0.14 * np.log10(sfr[i]) - 0.19 * (np.log10(M[i])) ** 2 + 0.12 * (np.log10(M[i]) * np.log10(sfr[i])) - 0.054 * (np.log10(sfr[i])) ** 2 - 12)
            metal = 10 ** (np.log10(solar_metal) + np.log10(OH) - np.log10(solar_OH))
            m_PISN = M_Z_star - alpha * np.log10(metal / Z_star)
            p_m1_args_new = [m_min, m_PISN, m_power]
            p_max = find_max(p_m1, m_min, m_PISN, p_m1_args_new)
            m1[i] = rejection_sampling(1, p_m1, m_min, m_max, p_max, p_m1_args_new)
            m2[i] = p_m2(m1[i], *p_m2_args)
        df.insert(df.shape[1], 'm1', m1)
        df.insert(df.shape[1], 'm2', m2)
    if p_m1_type == 'Z_given':
        # p_m1_args should be given in this order:
        # p_m1_args=[m_min,m_max,m_power,Z_star,M_Z_star,alpha]
        # see arXiv:2112.10256v2 for the description of above parameters.
        m_min, m_max, m_power, Z_star, M_Z_star, alpha = p_m1_args
        num = df.shape[0]
        m1 = np.zeros(num)
        m2 = np.zeros(num)
        metal = np.array(df['metal'])
        for i in range(num):
            m_PISN = M_Z_star - alpha * np.log10(metal[i] / Z_star)
            p_m1_args_new = [m_min, m_PISN, m_power]
            p_max = find_max(p_m1, m_min, m_PISN, p_m1_args_new)
            m1[i] = rejection_sampling(1, p_m1, m_min, m_max, p_max, p_m1_args_new)
            m2[i] = p_m2(m1[i], *p_m2_args)
        df.insert(df.shape[1], 'm1', m1)
        df.insert(df.shape[1], 'm2', m2)
    if p_m1_type == 'phys':
        # p_m1_args should be given in this order:
        # p_m1_args=[m_min,m_max,m_power,Z_star,M_Z_star,alpha,gamma,zeta]
        # see arXiv:2112.10256v2 for the description of above parameters.
        m_min, m_max, m_power, Z_star, M_Z_star, alpha, gamma, zeta = p_m1_args
        m = np.linspace(m_min, m_max, 100)  # an array for generating window functions for m1 to later multiply with source frame mass distribution, 100 resolution could be changed
        wins = [1 for _ in range(n_z)]
        z_arr = np.linspace(z_min, z_max, n_z)  # redshifts in which we calculate the window function
        # generating window functions
        for i in range(n_z):
            wins[i] = obs_win_normal(m, z_arr[i], Z_star, M_Z_star, alpha, gamma, zeta, t_min=100e6, kappa=1)
        num = df.shape[0]
        m1 = np.zeros(num)
        m2 = np.zeros(num)
        z = np.array(df['z'])  # getting actual redshifts of galaxies
        p_m1_args_new = [m_min, m_max, m_power]
        for i in range(num):
            p_s = p_m1(m, *p_m1_args_new)  # source frame mass distribution
            idx = find_nearest_idx(z_arr, z[i])  # finding the index of the nearest redshift in z_arr to the galaxy one
            p_o = p_s * wins[idx]  # observer frame mass distribution, no need to be normalized
            m1[i] = rejection_sampling_numeric(1, m, p_o)
            m2[i] = p_m2(m1[i], *p_m2_args)
        df.insert(df.shape[1], 'm1', m1)
        df.insert(df.shape[1], 'm2', m2)
    if p_m1_type == 'phen':
        # m_min and m_max should be given as the first and second argument of p_m1_args
        m_min = p_m1_args[0]
        m_max = p_m1_args[1]
        num = df.shape[0]
        m1 = np.zeros(num)
        m2 = np.zeros(num)
        p_max = find_max(p_m1, m_min, m_max, p_m1_args)
        for i in range(num):
            m1[i] = rejection_sampling(1, p_m1, m_min, m_max, p_max, p_m1_args)
            m2[i] = p_m2(m1[i], *p_m2_args)
        df.insert(df.shape[1], 'm1', m1)
        df.insert(df.shape[1], 'm2', m2)

In [ ]:
def generate_mask(cat, thresh=1, nside_mask=256, nside_map=512):
    """
    Generates a binary mask Healpix map given a catalog and threshold.
    This is used to mask out regions of the sky with too few sources, which can bias statistical analyses.
    
    Parameters:
        cat (DataFrame): Catalog containing 'ra' and 'dec' columns for right ascension and declination.
        thresh (int): Minimum number of sources required in a pixel to keep it in the mask.
        nside_mask (int): HEALPix nside parameter for the mask resolution.
        nside_map (int): HEALPix nside parameter for the map resolution.
    
    Returns:
        indices_to_mask (np.ndarray): Array of pixel indices at the map resolution that correspond to the mask.
        hpx_map (np.ndarray): HEALPix map where pixels with at least 'thresh' sources are set to 1, others are masked out (0).
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(cat.ra) * u.degree, dec=np.asarray(cat.dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value
    theta = np.radians(90. - b)
    phi = np.radians(l)
    npix = hp.nside2npix(nside_mask)
    
    # Assign each object to a HEALPix pixel at the mask resolution
    indices = hp.ang2pix(nside_mask, theta, phi)
    idx, counts = np.unique(indices, return_counts=True)
    
    # Initialize a HEALPix map at the map resolution
    hpx_map = np.zeros(npix)
    
    # Only keep pixels with at least 'thresh' sources; others are masked out
    thresh_idx = idx[counts >= thresh]
    hpx_map[thresh_idx] = 1  # 1 = keep, 0 = mask
    
    # For each object, also get its pixel index at the map resolution (for later masking)
    indices_to_mask = hp.ang2pix(nside_map, theta, phi)
    
    return indices_to_mask, hpx_map

def mask_cat(cat, thresh=1, nside_mask=256, nside_map=512):
    """
    Mask a catalog based on a threshold and HEALPix mask.
    Intended to remove sources in underpopulated sky regions, but function is incomplete in the original.
    
    Parameters:
        cat (DataFrame): Catalog containing 'ra' and 'dec' columns for right ascension and declination.
        thresh (int): Minimum number of sources required in a pixel to keep it in the mask.
        nside_mask (int): HEALPix nside parameter for the mask resolution.
        nside_map (int): HEALPix nside parameter for the map resolution.
        
    Returns:
        None: Function is incomplete and does not return anything.
    """
    # Convert RA/Dec to galactic coordinates
    c = SkyCoord(ra=np.asarray(cat.ra) * u.degree, dec=np.asarray(cat.dec) * u.degree, frame='icrs', unit='deg')
    l, b = c.galactic.l.value, c.galactic.b.value
    theta = np.radians(90. - b)
    phi = np.radians(l)
    npix = hp.nside2npix(nside_mask)
    pixel_indices = hp.ang2pix(nside_mask, theta, phi)
    # The following line references N_z, which is not defined in this function. This may need fixing.
    # N_bar = N_z / npix
    # print(N_z, npix, N_bar)
    # Function is incomplete in the original file. The intent is to mask out sources in low-density pixels.
    
def N_GW(z_min, z_max, N_bins, obs_time = 1, t_min = 500e6, kappa=1, grid_points=1999):
    """
    Calculate the expected number of gravitational wave events (N_GW) in each redshift bin.
    Parameters:
        z_min (float): Minimum redshift.
        z_max (float): Maximum redshift.
        N_bins (int): Number of redshift bins.
        obs_time (float): Observation time (default 1 year).
        t_min (float): Minimum time delay (default 500 Myr).
        kappa (float): Calibration factor (default 1).
        grid_points (int): Number of grid points for integration (default 1999).
    Returns:
        np.ndarray: Array with expected N_GW for each bin.
    """
    bin_size = (z_max - z_min) / N_bins
    N_GW_arr = []
    for i in range(N_bins):

        curr_N_GW = N_GW_bin(bin_size*i+z_min, bin_size*(i+1)+z_min, \
                             obs_time=obs_time, t_min=t_min, kappa=kappa, grid_points=grid_points)
        N_GW_arr.append(curr_N_GW)
    N_GW_arr = np.asarray(N_GW_arr)
    return(N_GW_arr)

def sample_sirens(gal_cat_bins,N_GW_arr ,p,p_args, SFR_Z_type,ampl = 1):
    '''
    Sample gravitational wave sirens from galaxy catalogs.
    Parameters:
        gal_cat_bins (list): List of galaxy catalogs for each redshift bin.
        N_GW_arr (np.ndarray): Expected number of GW events in each bin.
        p (function): Probability function for sampling.
        p_args (tuple): Arguments for the probability function.
        SFR_Z_type (str): Star formation rate and metallicity type.
        ampl (list): Amplitude factors for sampling (default [1]).
    Returns:
        list: Sampled galaxy catalogs for each bin.
    '''
    df_sampled = []
     
    for i in range(len(N_GW_arr)):
        df_sampled.append(g_selection_full(gal_cat_bins[i],int(N_GW_arr[i] * ampl[i] ),p ,p_args,SFR_Z_type))
     
    return df_sampled

def siren_catalog(df_filtered, z_min, z_max, N_bins, sky_frac,\
                  p, p_args,SFR_Z_type,\
                  p_m1_type,p_m1,p_m1_args, p_m2,p_m2_args,\
                  n_z,\
                  ampl=1, obs_time = 1, \
                  t_min = 500e6, kappa=1, grid_points=1999):
    '''
    Generate a catalog of gravitational wave sirens.
    Parameters:
        df_filtered (DataFrame): Filtered galaxy catalog.
        z_min (float): Minimum redshift.
        z_max (float): Maximum redshift.
        N_bins (int): Number of redshift bins.
        sky_frac (float): Fraction of the sky covered.
        p (function): Probability function for sampling.
        p_args (tuple): Arguments for the probability function.
        SFR_Z_type (str): Star formation rate and metallicity type.
        p_m1_type (str): Mass distribution type for primary mass.
        p_m1 (array): Primary mass distribution parameters.
        p_m1_args (tuple): Arguments for the primary mass distribution.
        p_m2 (array): Secondary mass distribution parameters.
        p_m2_args (tuple): Arguments for the secondary mass distribution.
        n_z (int): Number of redshift bins for mass distribution.
        ampl (list): Amplitude factors for sampling (default [1]).
        obs_time (float): Observation time (default 1 year).
        t_min (float): Minimum time delay (default 500 Myr).
        kappa (float): Calibration factor (default 1).
        grid_points (int): Number of grid points for integration (default 1999).
    Returns:
        tuple: Original and sampled galaxy catalogs for each bin.
    '''
    df_arr = []
    bin_size = (z_max - z_min) / N_bins
    
    for i in range(N_bins):
        df_arr.append(df_filtered[(df_filtered.z >= bin_size*i+z_min) & (df_filtered.z < bin_size*(i+1)+z_min)])
        
    N_GW_arr = N_GW(z_min, z_max, N_bins, obs_time = obs_time, \
                    t_min = t_min, kappa=kappa, grid_points=grid_points)* sky_frac
    
    df_sample_arr = sample_sirens(df_arr, N_GW_arr, p, p_args,SFR_Z_type, ampl=ampl)
    
    for i in range(len(df_sample_arr)):
        binary_masses_sample(df_sample_arr[i],p_m1_type,p_m1,p_m1_args,p_m2,p_m2_args,z_min,z_max,n_z,t_min,kappa)

    return df_arr, df_sample_arr#, (N_GW_arr * ampl).astype(int)

def siren_catalog_r(df,Random, z_min, z_max, N_bins, 
                  p, p_args,SFR_Z_type,
                  p_m1_type,p_m1,p_m1_args, p_m2,p_m2_args,
                  n_z,
                  ampl=500, obs_time = 1, 
                  t_min = 500e6, kappa=1, grid_points=1999):
    '''
    Generate a catalog of gravitational wave sirens with random sampling.
    Parameters:
        df (DataFrame): Galaxy catalog.
        Random (DataFrame): Randomly generated catalog for comparison.
        z_min (float): Minimum redshift.
        z_max (float): Maximum redshift.
        N_bins (int): Number of redshift bins.
        p (function): Probability function for sampling.
        p_args (tuple): Arguments for the probability function.
        SFR_Z_type (str): Star formation rate and metallicity type.
        p_m1_type (str): Mass distribution type for primary mass.
        p_m1 (array): Primary mass distribution parameters.
        p_m1_args (tuple): Arguments for the primary mass distribution.
        p_m2 (array): Secondary mass distribution parameters.
        p_m2_args (tuple): Arguments for the secondary mass distribution.
        n_z (int): Number of redshift bins for mass distribution.
        ampl (list): Amplitude factors for sampling (default [500]).
        obs_time (float): Observation time (default 1 year).
        t_min (float): Minimum time delay (default 500 Myr).
        kappa (float): Calibration factor (default 1).
        grid_points (int): Number of grid points for integration (default 1999).
    Returns:
        tuple: Original and sampled galaxy catalogs for each bin, with random samples.
    '''
    df_arr = []
    df_arr_ran=[]
    df_sample_ran_arr=[]
    bin_size = (z_max - z_min) / N_bins
    
    for i in range(N_bins):
        df_arr.append(df[(df.z >= bin_size*i+z_min) & (df.z < bin_size*(i+1)+z_min)])
        df_arr_ran.append(Random[(Random['z'] >= bin_size*i+z_min) & (Random['z'] < bin_size*(i+1)+z_min)])

    N_GW_arr = N_GW(z_min, z_max, N_bins, obs_time = obs_time, t_min = t_min, kappa=kappa, grid_points=grid_points)
    
    df_sample_arr = sample_sirens(df_arr, N_GW_arr, p, p_args,SFR_Z_type, ampl=ampl)
    #df_sample_ran_arr = sample_sirens(df_arr_ran, N_GW_arr*10*(ampl), p, p_args,SFR_Z_TYPE, ampl=ampl)

    for i in range(N_bins):
        df_sample_ran_arr.append((df_arr_ran[i].sample(n=int(N_GW_arr[i]*100*(ampl[i])),replace=False)))

    for i in range(len(df_sample_arr)):
        binary_masses_sample(df_sample_arr[i],p_m1_type,p_m1,p_m1_args,p_m2,p_m2_args,z_min,z_max,n_z,t_min,kappa)
    
    return df_arr, df_arr_ran, df_sample_arr,df_sample_ran_arr,(N_GW_arr).astype(int)

def rsd_p_gs(M,b_g,b_s,matter_pk,growthrate_s,kedges,sigmap_s):
    '''
    Calculate the redshift-space distortion power spectrum.
    Parameters:
        M (array): Mass array.
        b_g (array): Galaxy bias array.
        b_s (array): Spectroscopic bias array.
        matter_pk (array): Matter power spectrum.
        growthrate_s (array): Growth rate of structure.
        kedges (array): Edges of the k bins.
        sigmap_s (array): Sigma for the Gaussian smoothing.
    Returns:
        array: Redshift-space distortion power spectrum.
    '''
    return ((b_g*b_s*matter_pk)+(M**2*(b_g+b_s)*growthrate_s*matter_pk)+((M**4)*(growthrate_s**2)*matter_pk))*(1/(1+((kedges**2)*(M**2)*(sigmap_s**2)/2)))

In [ ]:
def to_wedges(self, muedges, ells=None):
    r"""Transform poles to wedges, with input :math:`\mu`-edges.
    Parameters
    ----------
    muedges : array
        :math:`\mu`-edges.
    ells : tuple, list, default=None
        Multipole orders to use in the Legendre expansion.
        If ``None``, all poles are used.
    Returns
    -------
    wedges : PowerSpectrumWedges
        Power spectrum wedges.
    """
    if ells is None:
        ells = self.ells
    elif np.ndim(ells) == 0:
        ells = [ells]
    muedges = np.array(muedges)
    mu = (muedges[:-1] + muedges[1:]) / 2.
    dmu = np.diff(muedges)
    edges = self.edges
    modes = (np.repeat(self.modes[0][:, None], len(dmu), axis=-1), np.repeat(mu[None, :], len(self.k), axis=0))
    power_nonorm, power_direct_nonorm = 0, 0
    for ell in ells:
        poly = np.diff(special.legendre(ell).integ()(muedges)) / dmu
        power_nonorm += self.power_nonorm[self.ells.index(ell), ..., None] * poly
        power_direct_nonorm += self.power_direct_nonorm[self.ells.index(ell), ..., None] * poly
    if 0 in self.ells:
        power_zero_nonorm = self.power_zero_nonorm[self.ells.index(0)]
    else:
        power_zero_nonorm = np.array(0., dtype=self.power_zero_nonorm.dtype)
    nmodes = self.nmodes[:, None] / dmu
    return PowerSpectrumWedges(edges, modes, power_nonorm, nmodes, wnorm=self.wnorm, shotnoise_nonorm=self.shotnoise_nonorm,
                               power_zero_nonorm=power_zero_nonorm, power_direct_nonorm=power_direct_nonorm,
                               attrs=self.attrs, mpicomm=getattr(self, 'mpicomm', None))
def shotnoise(self):
        """Normalized shot noise."""
        return self.shotnoise_nonorm / self.wnorm
    
def N_GW(z_min, z_max, N_bins, obs_time=1, t_min=500e6, kappa=1, grid_points=1999):
    '''
    Compute the expected number of gravitational wave (GW) events in each redshift bin.
    This is used to predict the GW event rate as a function of redshift, which is crucial for population studies and survey planning.
    '''
    bin_size = (z_max - z_min) / N_bins
    N_GW_arr = []
    for i in range(N_bins):
        # For each bin, integrate the merger rate over the redshift interval
        curr_N_GW = N_GW_bin(bin_size * i + z_min, bin_size * (i + 1) + z_min,
                             obs_time=obs_time, t_min=t_min, kappa=kappa, grid_points=grid_points)
        N_GW_arr.append(curr_N_GW)
    N_GW_arr = np.asarray(N_GW_arr)
    return N_GW_arr

def sample_sirens(gal_cat_bins, N_GW_arr, p, p_args, SFR_Z_type, ampl=1):
    '''
    Sample gravitational wave sirens from galaxy catalogs in each redshift bin.
    This simulates the process of selecting host galaxies for GW events, using a probability model for selection.
    '''
    df_sampled = []
    for i in range(len(N_GW_arr)):
        # For each bin, select the expected number of GW hosts using the provided probability model
        df_sampled.append(g_selection_full(gal_cat_bins[i], int(N_GW_arr[i] * ampl[i]), p, p_args, SFR_Z_type))
    return df_sampled

def siren_catalog(df_filtered, z_min, z_max, N_bins, sky_frac,
                  p, p_args, SFR_Z_type,
                  p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args,
                  n_z,
                  ampl=1, obs_time=1,
                  t_min=500e6, kappa=1, grid_points=1999):
    '''
    Generate a catalog of gravitational wave sirens (host galaxies) for a survey.
    This function bins the galaxy catalog in redshift, computes the expected GW event rate, samples hosts, and assigns binary masses.
    '''
    df_arr = []
    bin_size = (z_max - z_min) / N_bins
    # Bin the galaxy catalog by redshift
    for i in range(N_bins):
        df_arr.append(df_filtered[(df_filtered.z >= bin_size * i + z_min) & (df_filtered.z < bin_size * (i + 1) + z_min)])
    # Compute expected GW events per bin, scaled by sky coverage
    N_GW_arr = N_GW(z_min, z_max, N_bins, obs_time=obs_time,
                    t_min=t_min, kappa=kappa, grid_points=grid_points) * sky_frac
    # Sample host galaxies for each bin
    df_sample_arr = sample_sirens(df_arr, N_GW_arr, p, p_args, SFR_Z_type, ampl=ampl)
    # For each sampled host, assign binary component masses
    for i in range(len(df_sample_arr)):
        binary_masses_sample(df_sample_arr[i], p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args, z_min, z_max, n_z, t_min, kappa)
    return df_arr, df_sample_arr  # Also returns the original binned catalogs

def siren_catalog_r(df, Random, z_min, z_max, N_bins,
                  p, p_args, SFR_Z_type,
                  p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args,
                  n_z,
                  ampl=500, obs_time=1,
                  t_min=500e6, kappa=1, grid_points=1999):
    '''
    Generate a catalog of GW sirens with random sampling for comparison.
    This is useful for null tests and for understanding the impact of selection effects.
    '''
    df_arr = []
    df_arr_ran = []
    df_sample_ran_arr = []
    bin_size = (z_max - z_min) / N_bins
    # Bin both the real and random catalogs by redshift
    for i in range(N_bins):
        df_arr.append(df[(df.z >= bin_size * i + z_min) & (df.z < bin_size * (i + 1) + z_min)])
        df_arr_ran.append(Random[(Random['z'] >= bin_size * i + z_min) & (Random['z'] < bin_size * (i + 1) + z_min)])
    # Compute expected GW events per bin
    N_GW_arr = N_GW(z_min, z_max, N_bins, obs_time=obs_time,
                    t_min=t_min, kappa=kappa, grid_points=grid_points)
    # Sample hosts from the real catalog
    df_sample_arr = sample_sirens(df_arr, N_GW_arr, p, p_args, SFR_Z_type, ampl=ampl)
    # Sample random hosts for null test (oversample by 100x for statistics)
    for i in range(N_bins):
        df_sample_ran_arr.append((df_arr_ran[i].sample(n=int(N_GW_arr[i] * 100 * (ampl[i])), replace=False)))
    # Assign binary masses to the real sampled hosts
    for i in range(len(df_sample_arr)):
        binary_masses_sample(df_sample_arr[i], p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args, z_min, z_max, n_z, t_min, kappa)
    return df_arr, df_arr_ran, df_sample_arr, df_sample_ran_arr, (N_GW_arr).astype(int)

def rsd_p_gs(M, b_g, b_s, matter_pk, growthrate_s, kedges, sigmap_s):
    '''
    Calculate the redshift-space distortion (RSD) power spectrum for a galaxy sample.
    This is used to model the observed clustering of galaxies, accounting for peculiar velocities.
    '''
    # The formula combines galaxy bias, matter power spectrum, and growth rate, with Gaussian smoothing for small scales
    return ((b_g * b_s * matter_pk) + (M ** 2 * (b_g + b_s) * growthrate_s * matter_pk) + ((M ** 4) * (growthrate_s ** 2) * matter_pk)) * (1 / (1 + ((kedges ** 2) * (M ** 2) * (sigmap_s ** 2) / 2)))


# Preprocess data

In [ ]:
# Set the random seed for reproducibility of all random operations in this notebook.
np.random.seed(50)

In [ ]:
# Load the random catalog, main galaxy data, and k-correction data from FITS files.
# These files contain the SDSS random points, galaxy sample, and k-corrections for absolute magnitudes.
# The random catalog is used for statistical corrections and completeness estimation.
# The main data catalog contains the observed galaxies.
# The k-correct file is used to correct observed magnitudes to a common rest-frame.
# Note: File paths are specific to the data storage system in use.
#random=fits.open('/gpfs/dsadathosseini/sdss_random/random-0.dr72safe.fits')[1].data
random=fits.open('/gpfs/dsadathosseini/LSS2/sdss.physics.nyu.edu/lss/dr72/safe/Random_final/random/concatenated.fits')[1].data
data=fits.open('/gpfs/dsadathosseini/LSS2/sdss.physics.nyu.edu/lss/dr72/safe/0/post_catalog.dr72safe0.fits')[1].data
k_correct=fits.open('kcorrect.nearest.petro.z0.10.fits')[1].data

In [ ]:
# The following block (commented out) shows how to concatenate multiple FITS files into a single table.
# This is useful if the random catalog is split across many files and needs to be merged for analysis.
# It uses astropy's Table and vstack to combine all FITS tables in a directory tree.
# Uncomment and run if you need to regenerate the concatenated random catalog.

# from astropy.table import Table, vstack
# import os

# # Define the directory where the FITS files are located
# fits_dir = '/gpfs/dsadathosseini/LSS2/sdss.physics.nyu.edu/lss/dr72/safe/Random_final/random'

# # Initialize an empty table to hold the concatenated data
# concatenated_table = Table()

# # Loop over all files in the directory (and its subdirectories)
# for root, dirs, files in os.walk(fits_dir):
#     for file in files:
#         # Check if the file is a FITS file
#         if file.endswith('.fits'):
#             # Construct the full path to the file
#             file_path = os.path.join(root, file)
#             # Read the data from the FITS file into a table
#             print(file_path)
#             table = Table.read(file_path)
#             # Append the table to the concatenated table
#             concatenated_table = vstack([concatenated_table, table])

# # Write the concatenated table to a new FITS file
# concatenated_table.write('/gpfs/dsadathosseini/LSS2/sdss.physics.nyu.edu/lss/dr72/safe/Random_final/random/concatenated.fits', overwrite=True)

In [ ]:
# Compute the sky completeness map using the random catalog.
# This is used to correct for survey geometry and selection effects.
countsmap_random = countmap(np.array(random['RA']), np.array(random['DEC']), nside=256)
wheightedmap_random = weighted_countmap(np.array(random['RA']), np.array(random['DEC']), np.array(random['FGOT']), nside=256)
ave = wheightedmap_random / countsmap_random  # Average completeness per pixel

# Assign completeness to each galaxy based on its sky position
# This allows us to later weight galaxies by their local completeness

gal_indices = indices(np.array(data['RA']), np.array(data['DEC']), nside=256)
fgot_galaxy = ave[gal_indices]

# Compute galactic coordinates for each galaxy and random point
# These are used for spatial selection and for mapping the data on the sky
c = SkyCoord(np.array(data['RA']), np.array(data['DEC']), unit=(u.degree, u.degree))
latitude = c.galactic.b.value
longtitude = c.galactic.l.value

c1 = SkyCoord(np.array(random['RA']), np.array(random['DEC']), unit=(u.degree, u.degree))
latitude_r = c1.galactic.b.value
longtitude_r = c1.galactic.l.value

# Add new columns to the data tables for galactic coordinates and completeness
# This makes it easier to apply spatial cuts and weights later

data.Latitude = latitude
data.Longtitude = longtitude
data.FGOT_GALAXY = fgot_galaxy
random.Latitude = latitude_r
random.Longtitude = longtitude_r

# Compute color indices for the galaxy sample (g, r, and g-r)
# These are used for further selection and for stellar population studies
g_mag = np.array(data['ABSM'])[:, 1]
r_mag = np.array(data['ABSM'])[:, 2]
gr_mag = np.array(data['ABSM'])[:, 1] - np.array(data['ABSM'])[:, 2]


# Apply selection cuts to define the final galaxy and random samples
# These cuts remove galaxies with low completeness, outside the desired redshift range,
# and in problematic regions of the sky (e.g., near the Galactic plane or survey edges)
Data = data[(data['Z'] > 0.07) & (data['Z'] < 0.2) & (data.FGOT_GALAXY > 0.9) & (0 < data.Latitude) & ~((data.Latitude > 31) & (45 > data.Latitude) & (data.Longtitude > 75) & (97 > data.Longtitude))]
Random = random[(random['FGOT'] > 0.9) & (0 < random.Latitude) & ~((random.Latitude > 31) & (45 > random.Latitude) & (random.Longtitude > 75) & (97 > random.Longtitude))]

# Compute FKP weights for the galaxy sample
# These weights optimize the power spectrum measurement by downweighting regions with high density
P_FKP = 16000
Z_d = np.array(Data['Z'])
NZ_d = np.zeros(len(Z_d))
weight_FKP_d = np.zeros(len(Z_d))
for i in range(len(Z_d)):
    if ((Z_d[i] >= 0.17) & (Z_d[i] < 0.2)):
        n_z = 0.00286 - 0.0131 * Z_d[i]
        NZ_d[i] = n_z
        w_fkp = 1 / (1 + (P_FKP * n_z))
        weight_FKP_d[i] = w_fkp
    elif ((Z_d[i] > 0.07) & (Z_d[i] < 0.17)):
        n_z = 0.0014 * Z_d[i] + 0.00041
        NZ_d[i] = n_z
        w_fkp = 1 / (1 + (P_FKP * n_z))
        weight_FKP_d[i] = w_fkp
Data.NZ = NZ_d
Data.W_FKP = weight_FKP_d

# Assign random redshifts to the random catalog, drawn from the galaxy redshift distribution
# This is necessary for statistical corrections in clustering measurements
safe_z = Data['Z']
random_z = np.random.choice(safe_z, 33522980, replace=True)
Random.z = random_z

# Compute FKP weights for the random catalog using the same formula
P_FKP = 16000
Z_r = np.array(Random.z)
NZ_r = np.zeros(len(Z_r))
weight_FKP_r = np.zeros(len(Z_r))
for i in range(len(Z_r)):
    if ((Z_r[i] >= 0.17) & (Z_r[i] < 0.2)):
        n_z = 0.00286 - 0.0131 * Z_r[i]
        NZ_r[i] = n_z
        w_fkp = 1 / (1 + (P_FKP * n_z))
        weight_FKP_r[i] = w_fkp
    elif ((Z_r[i] > 0.07) & (Z_r[i] < 0.17)):
        n_z = 0.0014 * Z_r[i] + 0.00041
        NZ_r[i] = n_z
        w_fkp = 1 / (1 + (P_FKP * n_z))
        weight_FKP_r[i] = w_fkp
Random.NZ = NZ_r
Random.W_FKP = weight_FKP_r

# Assign weights to the galaxy and random samples
# For galaxies, use the inverse of the completeness as the weight
Data.WEIGHT = np.ones(335818)
Data.WEIGHT = 1. / data.FGOT_GALAXY
Random.WEIGHT = np.ones(33522980)

# Compute k-corrected magnitudes and colors for the galaxy sample
# These are used for stellar population and color-magnitude analyses
k_g_mag = np.array(k_correct['ABSMAG'])[:, 1]
k_r_mag = np.array(k_correct['ABSMAG'])[:, 2]
k_gr_mag = np.array(k_correct['ABSMAG'])[:, 1] - np.array(k_correct['ABSMAG'])[:, 2]

# Assign completeness and galactic coordinates to the k-corrected sample
k_gal_indices = indices(np.array(k_correct['RA']), np.array(k_correct['DEC']), nside=256)
fgot_k_galaxy = ave[k_gal_indices]
ck = SkyCoord(np.array(k_correct['RA']), np.array(k_correct['DEC']), unit=(u.degree, u.degree))
latitude_k = ck.galactic.b.value
longtitude_k = ck.galactic.l.value
k_correct.Latitude = latitude_k 
k_correct.Longtitude = longtitude_k
k_correct.FGOT_GALAXY = fgot_k_galaxy 

K_correct = k_correct[(k_correct['Z'] > 0.07) & (k_correct['Z'] < 0.2) & (k_correct.FGOT_GALAXY > 0.9) & (0 < k_correct.Latitude) & ~((k_correct.Latitude > 31) & (45 > k_correct.Latitude) & (k_correct.Longtitude > 75) & (97 > k_correct.Longtitude))]

In [ ]:
# Construct a table for the random catalog with relevant columns for clustering analysis.
# This table is converted to a pandas DataFrame for easier manipulation and compatibility with later analysis steps.
# The random catalog is used to model the survey selection function and correct for observational biases.
Random_a = Table([
    np.array(Random.RA),
    np.array(Random.DEC),
    np.array(Random.W_FKP),
    np.array(Random.MMAX),
    np.array(Random.NZ),
    np.array(Random.z),
    np.array(Random.WEIGHT)
], names=('RA', 'DEC', 'W_FKP', 'MMAX', 'NZ', 'z', 'Weight'))
Random_ap = Random_a.to_pandas()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
setup_logging()  # Turn on logging to screen for nbodykit and related libraries

# Set up the fiducial BOSS DR12 cosmology for all distance and clustering calculations
cosmolo = cosmology.Cosmology(h=0.7).match(Omega0_m=0.31)

# Add Cartesian position columns to the galaxy and random catalogs
# This is required for 3D clustering and power spectrum analysis
Data.Position = transform.SkyToCartesian(Data['RA'], Data['DEC'], Data['Z'], cosmo=cosmolo)
Random.Position = transform.SkyToCartesian(Random['RA'], Random['DEC'], Random.z, cosmo=cosmolo)

# Match each galaxy in the main sample to the nearest k-corrected galaxy in the sky
# This is used to assign stellar population properties (SFR, metallicity, mass) to each galaxy
cd = SkyCoord(ra=np.array(Data['RA']) * u.degree, dec=np.array(Data['DEC']) * u.degree)
ck = SkyCoord(ra=np.array(K_correct['RA']) * u.degree, dec=np.array(K_correct['DEC']) * u.degree)
idx, d2d, d3d = cd.match_to_catalog_sky(ck)

# Build a new table with matched properties for each galaxy
# SFR is computed from INTSFH and B1000, metallicity from METS, mass from MASS, and redshift from Z
# NZ and W_FKP are carried over for completeness and weighting
# The resulting DataFrame is the main input for further analysis

t1 = Table([
    np.array(K_correct[idx]['RA']),
    np.array(K_correct[idx]['DEC']),
    ((np.array(K_correct[idx]['INTSFH'])) * (np.array(K_correct[idx]['B1000']))) / (((0.67) ** 2) * 10 ** 9),
    np.array(K_correct[idx]['METS']),
    ((np.array(K_correct[idx]['MASS'])) * (1 / (10 ** (10) * (0.67) ** 2))),
    np.array(K_correct[idx]['Z']),
    np.array(Data.NZ),
    np.array(Data.W_FKP)
], names=('ra', 'dec', 'sfr', 'metal', 'M', 'z', 'NZ', 'W_FKP'))
df = t1.to_pandas()

# Filter the galaxy catalog to remove any entries with missing values
# This ensures that all subsequent analysis is performed on clean, complete data
df_filtered = df[df.z.notnull() & df.M.notnull() & df.ra.notnull() & df.dec.notnull()]

# Generating siren catalog using MZR host - galaxy probablity function / For examining GW bias dependency on Mk(pivot point in the probability function) 

In [ ]:
# This block generates the final siren catalogs and performs the main analysis loop for a range of M_c (pivot mass) values.
# The goal is to study how the GW host galaxy bias depends on the mass-metallicity relation (MZR) pivot point.

# Define the range of M_c values (pivot points for the probability function)
M_c_values = np.logspace(-1, 2, 15)
for i, M_c in enumerate(M_c_values):
    np.random.seed(i)  # Set seed for reproducibility for each M_c
    data_g = Data
    random_g = Random_ap
    K_correct_g = K_correct
    z_min = 0.07
    z_max = 0.20
    N_bins = 2  # Number of redshift bins
    m_bins = 1  # Number of mass bins (can be scalar or list of bin edges)
    p = P_MZR  # Probability function for host selection
    p_args = [4, .5, M_c, 0, 150]  # Arguments for the probability function
    SFR_Z_type = 'MZR'  # Use the MZR for host selection
    p_m1_type = 'Z_given'  # Use metallicity for primary mass selection
    p_m1 = m_kroupa  # Primary mass function
    p_m1_args = [5, 50, 2.3, 1e-4, 45, 1.5]  # Arguments for primary mass function
    p_m2 = sec_mass_sample_beta  # Secondary mass function
    p_m2_args = [5, 1]  # Arguments for secondary mass function
    n_z = 10  # Number of redshift grid points for window function
    ampl = 2000  # Amplitude factor for sampling
    obs_time = 1  # Observation time (years)
    t_min = 500e6  # Minimum time delay (years)
    kappa = 1  # Power-law index for delay time distribution
    grid_points = 1999  # Number of grid points for integration
    thresh = 15  # Threshold for masking
    nside_map = 512
    nside_mask = 256

    # Generate the siren and random catalogs for this M_c value
    # This uses the full selection, mass modeling, and sampling pipeline
    gal_cat_arr_k, gal_cat_ran_arr_k, siren_cat_arr_k, siren_cat_ran_arr_k, GW_Num_k = siren_catalog_r(
        df_filtered, Random_ap, z_min, z_max, N_bins,
        p, p_args, SFR_Z_type,
        p_m1_type, p_m1, p_m1_args, p_m2, p_m2_args,
        n_z,
        ampl=ampl, obs_time=obs_time,
        t_min=t_min, kappa=kappa, grid_points=grid_points)

    # Save the generated catalogs for later analysis
    Catalogs_M_c = siren_cat_arr_k, siren_cat_ran_arr_k
    np.save('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/Catalogs_M_c' + str(M_c), Catalogs_M_c)

    # Mass modeling: compute the chirp mass for each binary in the siren catalog
    for i in range(len(siren_cat_arr_k)):
        siren_cat_arr_k[i]['chirp_mass'] = chirp_mass(np.array(siren_cat_arr_k[i]['m1']), np.array(siren_cat_arr_k[i]['m2']))

    # Bin the siren catalog by chirp mass if needed
    if np.isscalar(m_bins):
        m2_min, m1_min, m_max = p_m2_args[0], p_m1_args[0], p_m1_args[1]
        cm_min, cm_max = chirp_mass(m2_min, m1_min), chirp_mass(m_max, m_max)
        m_bin_size = (cm_max - cm_min) / m_bins
        df_arr_k = [[1 for j in range(m_bins)] for i in range(N_bins)]
        for i in range(len(siren_cat_arr_k)):
            for j in range(m_bins):
                df_arr_k[i][j] = (siren_cat_arr_k[i][(siren_cat_arr_k[i].chirp_mass >= m_bin_size * j + cm_min)
                                                    & (siren_cat_arr_k[i].chirp_mass < m_bin_size * (j + 1) + cm_min)])
        siren_cat_arr_k = df_arr_k  # Overwrite with binned catalog
    else:
        # m_bins is a list of bin edges
        df_arr_k = [[1 for j in range(len(m_bins))] for i in range(N_bins)]
        for i in range(len(siren_cat_arr_k)):
            for j in range(len(m_bins)):
                df_arr_k[i][j] = (siren_cat_arr_k[i][(siren_cat_arr_k[i].chirp_mass >= m_bins[j][0])
                                                    & (siren_cat_arr_k[i].chirp_mass < m_bins[j][1])])
        siren_cat_arr_k = df_arr_k
    if np.isscalar(m_bins) == False:
        m_bins = len(m_bins)

    # Prepare catalogs for nbodykit power spectrum analysis
    # Convert pandas DataFrames to astropy Tables and then to ArrayCatalogs
    gal_cat_arr_cat_k = list(map(lambda x: Table.from_pandas(x), gal_cat_arr_k))
    gal_cat_ran_arr_cat_k = list(map(lambda x: Table.from_pandas(x), gal_cat_ran_arr_k))
    gal_cat_arr_cat1_k = list(map(lambda x: ArrayCatalog(x), gal_cat_arr_cat_k))
    gal_cat_ran_arr_cat1_k = list(map(lambda x: ArrayCatalog(x), gal_cat_ran_arr_cat_k))
    siren_cat_arr_cat_k = []
    siren_cat_ran_arr_cat_k = []
    siren_cat_arr_cat1_k = []
    siren_cat_ran_arr_cat1_k = []
    for i in range(len(siren_cat_arr_k)):
        siren_cat_arr_cat_k.append(list(map(lambda x: Table.from_pandas(x), siren_cat_arr_k[i])))
    for i in range(len(siren_cat_ran_arr_k)):
        siren_cat_ran_arr_cat_k.append(list(map(lambda x: Table.from_pandas(x), siren_cat_ran_arr2_k[i])))
    for i in range(len(siren_cat_arr_cat_k)):
        siren_cat_arr_cat1_k.append(list(map(lambda x: ArrayCatalog(x), siren_cat_arr_cat_k[i])))
    for i in range(len(siren_cat_ran_arr_cat_k)):
        siren_cat_ran_arr_cat1_k.append(list(map(lambda x: ArrayCatalog(x), siren_cat_ran_arr_cat_k[i])))

    # Assign 3D positions to all catalogs using the fiducial cosmology
    cosmolo = cosmology.Cosmology(h=0.7).match(Omega0_m=0.31)
    for i in range(len(gal_cat_arr_cat1_k)):
        gal_cat_arr_cat1_k[i]['Position'] = transform.SkyToCartesian(gal_cat_arr_cat1_k[i]['ra'], gal_cat_arr_cat1_k[i]['dec'], gal_cat_arr_cat1_k[i]['z'], cosmo=cosmolo)
    for i in range(len(gal_cat_ran_arr_cat1_k)):
        gal_cat_ran_arr_cat1_k[i]['Position'] = transform.SkyToCartesian(gal_cat_ran_arr_cat1_k[i]['RA'], gal_cat_ran_arr_cat1_k[i]['DEC'], gal_cat_ran_arr_cat1_k[i]['z'], cosmo=cosmolo)
    for i in range(N_bins):
        for j in range(m_bins):
            siren_cat_arr_cat1_k[i][j]['Position'] = transform.SkyToCartesian(siren_cat_arr_cat1_k[i][j]['ra'], siren_cat_arr_cat1_k[i][j]['dec'], siren_cat_arr_cat1_k[i][j]['z'], cosmo=cosmolo)
    for i in range(N_bins):
        for j in range(m_bins):
            siren_cat_ran_arr_cat1_k[i][j]['Position'] = transform.SkyToCartesian(siren_cat_ran_arr_cat1_k[i][j]['RA'], siren_cat_ran_arr_cat1_k[i][j]['DEC'], siren_cat_ran_arr_cat1_k[i][j]['z'], cosmo=cosmolo)

    #power of galaxies

    P0 = 1e4
    kmin=0.0
    kmax=0.5
    dk=0.02
    kbin=int((kmax-kmin)/dk)
    kedges = np.linspace(kmin, kmax, kbin+1)
    gal_pk_pypower=np.zeros((N_bins, kbin))
    gal_pk_pypower_sn = np.zeros((N_bins))
    matter_g_pk_pypower=np.zeros((N_bins,kbin))
    FKP_gal_pypower=np.zeros(N_bins)
    C_g_k=np.zeros((N_bins,kbin))
    sigma_p_g=np.zeros((N_bins))
    f_g=np.zeros((N_bins))
    data=gal_cat_arr_cat1_k
    randoms=gal_cat_ran_arr_cat1_k
    P_g_rsd=np.zeros((N_bins))

    # Loop over redshift bins to compute galaxy power spectrum and related quantities
    for i in range(N_bins):
        result_G = {}
        zbin_names = ['zbin1', 'zbin2']

        ells = (0, 2, 4)
        data=gal_cat_arr_cat1_k
        randoms=gal_cat_ran_arr_cat1_k
        # Define redshift interval and box size for each bin (survey geometry)
        if i==0 :
            interval=np.linspace(0.07, 0.135, 100)
            BoxSize = [272., 535., 300.]
            Boxcenter = [-159.74, -11.47, 127.62]
        elif i==1: 
            interval=np.linspace(0.135, 0.2, 100)
            BoxSize = [544., 1040., 585.]
            Boxcenter = [-304.22, -21.37, 247.3] 


        FSKY = 6813./41253. # a made-up value
        # Compute redshift histogram for randoms to estimate selection function
        zhist_g_i_k = RedshiftHistogram(gal_cat_ran_arr_cat1_k[i], FSKY, cosmolo, redshift='z',bins = interval)
        alpha_g_i_k = 1.0 * gal_cat_arr_cat1_k[i].csize / gal_cat_ran_arr_cat1_k[i].csize
        # Interpolate n(z) for data and randoms
        nz_g_i = np.interp(np.array( gal_cat_arr_cat1_k[i]['z']),zhist_g_i_k.bin_centers,alpha_g_i_k*zhist_g_i_k.nbar)
        nz_gr_i = np.interp(np.array(gal_cat_ran_arr_cat1_k[i]['z']),zhist_g_i_k.bin_centers,alpha_g_i_k*zhist_g_i_k.nbar)
        # Build FKP catalog with weights for optimal power spectrum estimation
        fkp_gal_i = FKPCatalog(gal_cat_arr_cat1_k[i], gal_cat_ran_arr_cat1_k[i])
        fkp_gal_i['randoms/NZ'] = nz_gr_i
        fkp_gal_i['data/NZ'] = nz_g_i
        fkp_gal_i['data/FKPWeight'] = 1.0 / (1 + fkp_gal_i['data/NZ'] * P0 )
        fkp_gal_i['randoms/FKPWeight'] = 1.0 / (1 + fkp_gal_i['randoms/NZ'] * P0 )
        # Compute weighted mean redshift for the bin
        FKP_gal_pypower[i]=np.sum(np.array(fkp_gal_i['data/FKPWeight']) * np.array(gal_cat_arr_cat1_k[i]['z']))/np.sum(np.array(fkp_gal_i['data/FKPWeight']))

        # Compute the power spectrum using FFT-based estimator
        result_g_i = CatalogFFTPower(data_positions1=np.array(gal_cat_arr_cat1_k[i]['Position']), data_weights1=np.array(fkp_gal_i['data/FKPWeight']),
                                 randoms_positions1=np.array(gal_cat_ran_arr_cat1_k[i]['Position']), randoms_weights1=np.array(fkp_gal_i['randoms/FKPWeight']),
                                 edges=kedges, ells=ells, interlacing=2, boxsize=None,boxcenter=None, nmesh=256, resampler='tsc',
                                 los='firstpoint', position_type='pos', mpiroot=0)
        # Save the result for later analysis
        result_g_i.save('/gpfs/dsadathosseini/sdss_mzsfr_newomcorr_deltah'+str(deltah)+'_Z002_sfr372_eph05_zetal05_zetah025_file_final/result_g_Mc'+str(M_c)+'_i'+str(i)) 


        Nmu = 1
        mu_edges = numpy.linspace(0, 1, Nmu+1)
        poles_g_i = result_g_i.poles
        wedges_g_i = poles_g_i.to_wedges(muedges=mu_edges, ells=None)
        gal_pk_pypower[i,:]=wedges_g_i.power[:,0]



        gal_pk_pypower_sn[i]=shotnoise(wedges_g_i)
        c =cosmology.Planck15
        sigma_v=400
        zeff_g_i=np.sum(np.array(gal_cat_arr_cat1_k[i]['z'] * np.array(fkp_gal_i['data/FKPWeight'])))/np.sum(np.array(fkp_gal_i['data/FKPWeight']))

        sigma_p_g[i]=0.88*(sigma_v/100)*((1+zeff_g_i)/4)**(-0.4)
        f_g[i]=cosmo.Om(zeff_g_i)**0.55
        # Compute nonlinear matter power spectrum using Halofit
        pnl_g_i=cosmology.HalofitPower(c, redshift=zeff_g_i)
        matter_g_pk_pypower[i,:]=pnl_g_i(0.5 * (kedges[1:] + kedges[:-1]))
        # Compute survey volume for normalization
        chi_max_k=cosmolo.comoving_distance(np.max(np.array(gal_cat_arr_cat1_k[i]['z'])))*cosmolo.H0/100
        chi_min_k=cosmolo.comoving_distance(np.min(np.array(gal_cat_arr_cat1_k[i]['z'])))*cosmolo.H0/100
        V_w_k= (4 * np.pi/3) * ((6813)/41253) * (chi_max_k**3-chi_min_k**3)
        # Compute covariance for the galaxy power spectrum
        C_g_k[i,:]=(1/(V_w_k))*(((2*np.pi)**3)/(4*np.pi*((0.5 * (kedges[1:] + kedges[:-1]))**2)*dk))*(2*(gal_pk_pypower[i,:]+gal_pk_pypower_sn[i])**2)


    gal_cat_pk_k_flat = gal_pk_pypower.flatten()
    gal_cat_pk_k_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/gal_pk_pypower_M_c'+str(M_c))     
    
    C_g_k_flat = C_g_k.flatten()
    C_g_k_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/C_g_k_M_c'+str(M_c))     
    


    #power of sirens



    siren_pk_pypower=np.zeros((N_bins,m_bins, kbin))
    siren_pk_pypower_sn = np.zeros((N_bins,m_bins))
    FKP_siren_pypower=np.zeros((N_bins,m_bins))
    power_gs=np.zeros((N_bins,m_bins,kbin))
    bias_error=np.zeros((N_bins,m_bins,kbin))
    C_s_k=np.zeros((N_bins,m_bins,kbin))
    C_gs_k=np.zeros((N_bins,m_bins,kbin))
    f_s=np.zeros((N_bins,m_bins))
    data=siren_cat_arr_cat1_k
    randoms=siren_cat_ran_arr_cat1_k
    p_gg_numeric=np.zeros((N_bins, kbin))
    p_ss_numeric=np.zeros((N_bins,m_bins,kbin))
    b_gg_numeric=np.zeros((N_bins, kbin))
    b_gm_rsd=np.zeros((N_bins, kbin,2))
    b_ss_numeric=np.zeros((N_bins,m_bins,kbin))
    bias_fit_g=np.zeros((N_bins,kbin))
    bias_fit_s=np.zeros((N_bins,m_bins,kbin))
    Z_eff = np.zeros((N_bins,m_bins))
    for i in range(N_bins):
        for j in range(m_bins):


            ells = (0, 2, 4)
            data=siren_cat_arr_cat1_k
            randoms=siren_cat_ran_arr_cat1_k
            if i==0 :
                interval=np.linspace(0.07, 0.135, 100)
                BoxSize = [272., 535., 300.]
                Boxcenter = [-159.74, -11.47, 127.62]
            elif i==1: 
                interval=np.linspace(0.135, 0.2, 100)
                BoxSize = [544., 1040., 585.]
                Boxcenter = [-304.22, -21.37, 247.3] 

            FSKY = 6813./41253. # a made-up value
            # Compute redshift histogram for randoms to estimate selection function
            zhist_g_i_k = RedshiftHistogram(gal_cat_ran_arr_cat1_k[i], FSKY, cosmolo, redshift='z',bins = interval)
            zhist_s_i_j_k = RedshiftHistogram(siren_cat_ran_arr_cat1_k[i][j], FSKY, cosmolo, redshift='z',bins = interval)
            alpha_s_i_j_k = 1.0 * siren_cat_arr_cat1_k[i][j].csize / siren_cat_ran_arr_cat1_k[i][j].csize
            # Interpolate n(z) for siren data and randoms
            nz_s_i_j = np.interp(np.array( siren_cat_arr_cat1_k[i][j]['z']),zhist_g_i_k.bin_centers,alpha_s_i_j_k*zhist_g_i_k.nbar)
            nz_sr_i_j = np.interp(np.array(siren_cat_ran_arr_cat1_k[i][j]['z']),zhist_g_i_k.bin_centers,alpha_s_i_j_k*zhist_g_i_k.nbar)
            # Build FKP catalog with weights for optimal power spectrum estimation
            fkp_s_i_j = FKPCatalog(siren_cat_arr_cat1_k[i][j], siren_cat_ran_arr_cat1_k[i][j])
            fkp_s_i_j['randoms/NZ'] = nz_sr_i_j
            fkp_s_i_j['data/NZ'] = nz_s_i_j
            fkp_s_i_j['data/FKPWeight'] = 1.0 / (1 + fkp_s_i_j['data/NZ'] * 1e4)
            fkp_s_i_j['randoms/FKPWeight'] = 1.0 / (1 + fkp_s_i_j['randoms/NZ'] * 1e4)
            # Compute weighted mean redshift for the siren bin
            FKP_siren_pypower[i,j]=np.sum(np.array(fkp_s_i_j['data/FKPWeight']) * np.array(siren_cat_arr_cat1_k[i][j]['z']))/np.sum(np.array(fkp_s_i_j['data/FKPWeight']))

            # Compute the power spectrum using FFT-based estimator for sirens
            result_s_i_j = CatalogFFTPower(data_positions1=np.array(siren_cat_arr_cat1_k[i][j]['Position']), data_weights1=np.array(fkp_s_i_j['data/FKPWeight']),
                                     randoms_positions1=np.array(siren_cat_ran_arr_cat1_k[i][j]['Position']), randoms_weights1=np.array(fkp_s_i_j['randoms/FKPWeight']),
                                     edges=kedges, ells=ells, interlacing=2, boxsize=None,boxcenter=None, nmesh=256, resampler='tsc',
                                     los='firstpoint', position_type='pos', mpiroot=0)
            # Save the result for later analysis
            result_s_i_j.save('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/result_S_Mc'+str(M_c)+'_i'+str(i)+'_j'+str(j)) 

            Nmu = 1
            mu_edges = numpy.linspace(0, 1, Nmu+1)
            poles_s_i_j = result_s_i_j.poles
            wedges_s_i_j= poles_s_i_j.to_wedges(muedges=mu_edges, ells=None)
            siren_pk_pypower[i,j]=wedges_s_i_j.power[:,0]



            siren_pk_pypower_sn[i,j]=shotnoise(wedges_s_i_j)
            sigma_v=400
            lower_limit_M = 0
            upper_limit_M = 1
            c = cosmology.Planck15
            
            zeff_s_ij=np.sum(np.array(siren_cat_arr_cat1_k[i][j]['z'] * np.array(fkp_s_i_j['data/FKPWeight'])))/np.sum(np.array(fkp_s_i_j['data/FKPWeight']))
            Z_eff[i][j]=zeff_s_ij
            Z_eff_flat = Z_eff.flatten()
            Z_eff_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/Z_eff_Mc'+str(M_c))
 
            
            f_s[i,j]=cosmo.Om(zeff_s_ij)**0.55

            sigma_p_s_ij=0.88*(sigma_v/100)*((1+zeff_s_ij)/4)**(-0.4)
            b_lin_g_i=np.sqrt(gal_pk_pypower[i]/matter_g_pk_pypower[i])
            b_lin_s_ij=np.sqrt(siren_pk_pypower[i,j]/matter_g_pk_pypower[i])

            for k in range(kbin):
                b_g = np.linspace(0,5,120)
                b_s = np.linspace(0,5,120)
                M = np.linspace(0, 1, 100)
                dM = M[1]-M[0]


                ratio_g=gal_pk_pypower[i][k]/matter_g_pk_pypower[i][k]
                ratio_s=siren_pk_pypower[i][j][k]/matter_g_pk_pypower[i][k]

                integrand_g = (b_g**2+2*b_g*f_g[i]*M[:,np.newaxis]**2+f_g[i]**2*M[:,np.newaxis]**4)*(1/(1+((0.5 * (kedges[1:] + kedges[:-1])[k])**2*M[:,np.newaxis]**2*sigma_p_g[i]**2/2)))
                integrand_s = (b_s**2+2*b_s*f_s[i,j]*M[:,np.newaxis]**2+f_s[i,j]**2*M[:,np.newaxis]**4)*(1/(1+((0.5 * (kedges[1:] + kedges[:-1])[k])**2*M[:,np.newaxis]**2*sigma_p_s_ij**2/2)))

                integral_g = np.sum(integrand_g * dM, axis=0)
                integral_s = np.sum(integrand_s * dM, axis=0)

                bias_fit_g[i][k]=b_g[np.argmin((ratio_g-integral_g)**2)]
                bias_fit_s[i][j][k]=b_s[np.argmin((ratio_s-integral_s)**2)]

                power_gs[i][j][k]=integrate.quad(rsd_p_gs,0,1,args=(bias_fit_g[i][k],bias_fit_s[i][j][k],matter_g_pk_pypower[i][k],f_s[i,j],0.5 * (kedges[1:] + kedges[:-1])[k],sigma_p_s_ij))[0]




            p_gg_numeric[i]=bias_fit_g[i]*matter_g_pk_pypower[i]
            p_ss_numeric[i][j]=bias_fit_s[i][j]*matter_g_pk_pypower[i]


            chi_max_g=cosmolo.comoving_distance(np.max(np.array(gal_cat_arr_cat1_k[i]['z'])))*cosmolo.H0/100
            chi_min_g=cosmolo.comoving_distance(np.min(np.array(gal_cat_arr_cat1_k[i]['z'])))*cosmolo.H0/100
            V_w_k_g= (4 * np.pi/3) * ((6813)/41253) * (chi_max_g**3-chi_min_g**3)

            C_g_k[i,:]=(1/(V_w_k_g))*(((2*np.pi)**3)/(4*np.pi*((0.5 * (kedges[1:] + kedges[:-1]))**2)*dk))*(2*(gal_pk_pypower[i,:]+gal_pk_pypower_sn[i])**2)


            chi_maxs_s=cosmolo.comoving_distance(np.max(np.array(siren_cat_arr_cat1_k[i][j]['z'])))*cosmolo.H0/100
            chi_mins_s=cosmolo.comoving_distance(np.min(np.array(siren_cat_arr_cat1_k[i][j]['z'])))*cosmolo.H0/100
            V_w_k_s= (4 * np.pi/3) * ((6813)/41253) * (chi_maxs_s**3-chi_mins_s**3)
            C_s_k[i,j,:]=(1/(V_w_k_s))*(((2*np.pi)**3)/(4*np.pi*((0.5 * (kedges[1:] + kedges[:-1]))**2)*dk))*(2*(siren_pk_pypower[i][j]+siren_pk_pypower_sn[i,j])**2)



            C_gs_k[i,j,:] = (1/(V_w_k))*(((2*np.pi)**3)/(4*np.pi*((0.5 * (kedges[1:] + kedges[:-1]))**2)*dk))* (2 * (power_gs[i,j] + gal_pk_pypower_sn[i])**2)


            bias_error[i,j,:]=np.sqrt((1/(gal_pk_pypower[i])**2)*(C_s_k[i,j])+((siren_pk_pypower[i][j]/(gal_pk_pypower[i])**2)**2)*(C_g_k[i,:])-2*((siren_pk_pypower[i][j])/((gal_pk_pypower[i])**3))*C_gs_k[i,j,:])


    siren_cat_pk_k_flat = siren_pk_pypower.flatten()
    siren_cat_pk_k_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/siren_pk_pypower_M_c'+str(M_c))
    bias_error_flat = bias_error.flatten()
    bias_error_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/bias_error_M_c'+str(M_c))
    C_s_k_flat = C_s_k.flatten()
    C_s_k_flat.tofile('/gpfs/dsadathosseini/sdss_MZR_s50_dl4_dh05_newomcorr_file_final/C_s_k_M_c'+str(M_c))
